In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import ipywidget_pivot_table as pt

## Demo notebook for `jupyter_widget_pivot_table`
+ This [Jupyter widget](https://ipywidgets.readthedocs.io/en/stable/#) wraps the very convenient [pivotTable.js lib](https://pivottable.js.org/examples/)
+ The examples below are reproduced from the pivotTable.js [examples page](https://pivottable.js.org/examples/)
+ `.Pivot()` corresponds to pivotTable.js [`pivot()` method](https://github.com/nicolaskruchten/pivottable/wiki/Parameters#pivotinput-options-locale)
+ `.PivotUI()` corresponds to pivotTable.js [`pivotUI()` method](https://github.com/nicolaskruchten/pivottable/wiki/Parameters#pivotuiinput-options-overwrite-locale)
+ data must be input as a [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)
+ options must be input as a `Pivot_Options` or `PivotUI_Options` object.
    + key-values as per the pivotTable.js [documentation](https://github.com/nicolaskruchten/pivottable/wiki/Parameters)
    + These objects have first level autocomplete.
+ A `PivotUI` object p (say `p` for example) has 2 buttons: **Save** and **Restore**.
    + **Save** to snapshot the current configuration into dataframe `p.table.df_export` and dict `p.table.options`.  
    _Note_: Options that are javascript functions are discarded when passed to Python
    + **Restore** to apply the options saved

## 1 - pivot - Base
+ Cf. original in [jsfiddle](https://jsfiddle.net/nicolaskruchten/kn381h7s/)

In [3]:
df = pt.samples.df_tips
dic = {
    'sumOverSum': '$.pivotUtilities.aggregators["Sum over Sum"]',
    'heatmap': '$.pivotUtilities.renderers["Heatmap"]'    
}

p = pt.Pivot(df_data=df)
opts = p.options
opts.rows = ["sex", "smoker"]
opts.cols = ["day", "time"]
opts.vals = ["tip", "total_bill"]
opts.aggregator  = '{sumOverSum}(["tip", "total_bill"])'.format(**dic)
opts.renderer = '{heatmap}'.format(**dic)

p

Pivot()

## 1 bis - pivot - Base
+ Cf. [original in pivotTable.js example](https://pivottable.js.org/examples/scatter.html)

In [4]:
df = pt.samples.df_iris

p = pt.Pivot(df_data=df)

opts = p.options
opts.rows = ["Petal.Length"]
opts.cols = ["Petal.Width", "Species"]
opts.renderer = '$.pivotUtilities.c3_renderers["Scatter Chart"]'
opts.rendererOptions = {
    'c3': {
        'size': {
            'width': 700, 'height': 500
        }
    }
}
p

Pivot()

## 2 - pivotUI - Base
+ Options as `PivotUI_Options` helper object
+ Cf. [original jsfiddle](https://jsfiddle.net/nicolaskruchten/kn381h7s/)

In [5]:
df = pt.samples.df_tips

p = pt.PivotUI(df_data=df)
opts = p.options

dic = {
    'sumOverSum': '$.pivotUtilities.aggregators["Sum over Sum"]',
    'heatmap': '$.pivotUtilities.renderers["Heatmap"]'    
}
opts.rows = ['sex', 'smoker']
opts.cols = ['day', 'time']
opts.vals = ['tip', 'total_bill']
opts.aggregatorName = 'Sum over Sum'
opts.rendererName = 'Heatmap'

p

PivotUI()

## 3 - pivotUI - derived attributes
+ Cf. original in [pivotTable.js example](https://pivottable.js.org/examples/mps.html)

In [6]:
df = pt.samples.df_mps

p = pt.PivotUI(df_data=df)
opts = p.options

opts.rows = ['Gender Imbalance']
opts.cols = ['Age Bin']
opts.derivedAttributes = {
    'Age Bin': '$.pivotUtilities.derivers.bin("Age", 10)',
    'Gender Imbalance': 'function(mp) { return mp["Gender"] == "Male" ? 1 : -1; }'    
}

p

PivotUI()

## 5 - pivotUI - prepopulated with click callback
+ Cf. [original pivotTable.js example](https://pivottable.js.org/examples/mps_prepop.html)

In [7]:
df = pt.samples.df_mps

p = pt.PivotUI(df_data=df)
opts = p.options

opts.rows = ["Province"]
opts.cols = ["Party"]
opts.aggregatorName = "Integer Sum"
opts.vals = ["Age"]
opts.rendererName = "Heatmap"
opts.rendererOptions = {
    'table': {
        'clickCallback': """function(e, value, filters, pivotData){
                            var names = [];
                            pivotData.forEachMatchingRecord(filters,
                                function(record){ names.push(record.Name); });
                            alert(names.join(\"\\n\"));
                        }"""
    }
}

print('-> Click on a cell to view the contents')
p

-> Click on a cell to view the contents


PivotUI()

## 6 - pivotUI - custom aggregators and sort order
+ Cf. [original pivotTable.js example](https://pivottable.js.org/examples/mps_agg.html)

In [8]:
df = pt.samples.df_mps

p = pt.PivotUI(df_data=df)
opts = p.options

dic = {
    'tpl': '$.pivotUtilities.aggregatorTemplates',
    'sortAs': '$.pivotUtilities.sortAs'
}

opts.rows = ["Province"]
opts.cols = ["Party"]
opts.aggregators = {
    "Number of MPs":      'function() {{ return {tpl}.count()() }}'.format(**dic),
    "Average Age of MPs": 'function() {{ return {tpl}.average()(["Age"]) }}'.format(**dic)
}
opts.sorters = {
    'Age': 'function(a,b){ return b-a; }', # sort backwards
    'Province': """{sortAs}(["British Columbia", "Alberta", "Saskatchewan", "Manitoba",
                             "Territories", "Ontario", "Quebec", "New Brunswick",
                             "Prince Edward Island", "Nova Scotia",
                             "Newfoundland and Labrador"])""".format(**dic)
}

p

PivotUI()

## 7 - pivotUI - C3 chart renderer
+ Cf. original in [pivotTable.js example](https://pivottable.js.org/examples/c3.html)

In [9]:
df = pt.samples.df_mps

p = pt.PivotUI(df_data=df)
opts = p.options

opts.rows = ["Province"]
opts.cols = ["Party"]
opts.renderers = '$.extend($.pivotUtilities.renderers, $.pivotUtilities.c3_renderers)'
opts.rendererName = "Horizontal Stacked Bar Chart"
opts.rowOrder = "value_z_to_a"
opts.colOrder = "value_z_to_a"
opts.rendererOptions = {
    'c3': {
        'data': {
            'colors': {
                'Liberal': '#dc3912',
                'Conservative': '#3366cc',
                'NDP': '#ff9900',
                'Green':'#109618',
                'Bloc Quebecois': '#990099'
            }
        }
    }
}

p

PivotUI()

## 8 - pivotUI - D3 chart renderer
+ Cf. original in [pivotTable.js example](https://pivottable.js.org/examples/d3.html)

In [10]:
df = pt.samples.df_mps

p = pt.PivotUI(df_data=df)
opts = p.options

opts.rows = ["Province", "Party"]
opts.cols = []
opts.rendererName = "Treemap"

p

PivotUI()

## 9 - pivotUI - date derivers, sort overriding and custom heatmap colours
+ Cf. original in [pivotTable.js example](https://pivottable.js.org/examples/montreal_2014.html)

In [11]:
df = pt.samples.df_weather

p = pt.PivotUI(df_data=df)
opts = p.options

putil = '$.pivotUtilities'
dic = {
    'dateFormat': putil+'.derivers.dateFormat',
    'sortAs': putil+'.sortAs',
    'tpl': putil+'.aggregatorTemplates',
    'numberFormat': putil+".numberFormat({ suffix: ' °C' })"    
}
dic['putil'] = putil

opts.hiddenAttributes = ['Date', 'Max Temp (C)', 'Mean Temp (C)',
                        'Min Temp (C)', 'Total Rain (mm)', 'Total Snow (cm)']

opts.derivedAttributes = {
    'month name': "{dateFormat}('Date', '%n', true)".format(**dic),
    'day name': "{dateFormat}('Date', '%w', true)".format(**dic)
}

opts.rows = ['day name']
opts.cols = ['month name']
opts.sorters = {
    'month name': """{sortAs}(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])""".format(**dic),
    'day name': "{sortAs}(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])".format(**dic)
}
opts.aggregators = {
    'Mean Temperature': "function () {{ return {tpl}.average({numberFormat})(['Mean Temp (C)']) }}".format(**dic),
    'Max Temperature': "function () {{ return {tpl}.max({numberFormat})(['Max Temp (C)']) }}".format(**dic),
    'Min Temperature': "function () {{ return {tpl}.min({numberFormat})(['Min Temp (C)']) }}".format(**dic)
}
opts.renderers = '$.extend({putil}.renderers, {putil}.c3_renderers, {putil}.export_renderers)'.format(**dic)
opts.rendererName = 'Heatmap'
opts.rendererOptions = {
    'heatmap': {
        'colorScaleGenerator': """function (values) {
            return d3.scale.linear()
                .domain([-35, 0, 35])
                .range(['#77F', '#FFF', '#F77'])
        }"""
    }
}

p

PivotUI()

### Saved dataframe

+ Snapshot by Save button (first click a few milliseconds post creation)
+ Multi-index corresponding to pivot table


In [12]:
# p.df_export

### Modify options properties

+ Triggers re-rendering of all views

In [13]:
p.options.aggregatorName = 'Mean Temperature'

In [14]:
p.options.aggregators = {
    'Mean Temperature': "function () {{ return {tpl}.average({numberFormat})(['Mean Temp (C)']) }}".format(**dic),
    'Max Temperature': "function () {{ return {tpl}.max({numberFormat})(['Max Temp (C)']) }}".format(**dic)
}

In [15]:
opts.rendererOptions = {
    'heatmap': {
        'colorScaleGenerator': """function (values) {
            return d3.scale.linear()
                .domain([-20, 0, 20])
                .range(['#77F', '#FFF', '#F77'])
        }"""
    }
}